<a href="https://colab.research.google.com/github/Black3rror/AI/blob/master/CV_course_HW3_(SVHN)_my_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

from scipy.io import loadmat
import urllib.request    # to download SVHN dataset

NAME = "SVHN-CNN"

In [8]:
%load_ext tensorboard
from tensorflow.keras.callbacks import TensorBoard
import datetime   # to organize TensorBoard files

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
trainURL = 'http://ufldl.stanford.edu/housenumbers/train_32x32.mat'
urllib.request.urlretrieve(trainURL, 'train_32x32.mat')
testURL = 'http://ufldl.stanford.edu/housenumbers/test_32x32.mat'
urllib.request.urlretrieve(testURL, 'test_32x32.mat')

('test_32x32.mat', <http.client.HTTPMessage at 0x7fc06249b3c8>)

In [10]:
trainSet = loadmat('train_32x32.mat')
x_train_raw = trainSet['X']
y_train = trainSet['y']

testSet = loadmat('test_32x32.mat')
x_test_raw = testSet['X']
y_test = testSet['y']

x_train = np.transpose(x_train_raw, (3, 0, 1, 2))
x_test = np.transpose(x_test_raw, (3, 0, 1, 2))
y_train = to_categorical(y_train - 1)
y_test = to_categorical(y_test - 1)

print(x_train.shape)
print(y_train.shape)

x_train = x_train/255.0
x_test = x_test/255.0

(73257, 32, 32, 3)
(73257, 10)


In [11]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

In [12]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, callbacks=[tensorboard_callback])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 256)       7168      
_________________________________________________________________
activation_4 (Activation)    (None, 30, 30, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 256)       590080    
_________________________________________________________________
activation_5 (Activation)    (None, 13, 13, 256)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)             

In [ ]:
%tensorboard --logdir logs/fit    # to run the TensorBoard in the notebook